In [2]:
from picamera2 import Picamera2
from PIL import Image
import numpy as np
from tflite_runtime.interpreter import Interpreter

In [3]:
picam2 = Picamera2()

[0:53:06.217228782] [5365]  INFO Camera camera_manager.cpp:284 libcamera v0.1.0+118-563cd78e
[0:53:06.263018647] [5380]  WARN RPiSdn sdn.cpp:39 Using legacy SDN tuning - please consider moving SDN inside rpi.denoise
[0:53:06.265797243] [5380]  INFO RPI vc4.cpp:444 Registered camera /base/soc/i2c0mux/i2c@1/ov5647@36 to Unicam device /dev/media1 and ISP device /dev/media0
[0:53:06.265860039] [5380]  INFO RPI pipeline_base.cpp:1142 Using configuration file '/usr/share/libcamera/pipeline/rpi/vc4/rpi_apps.yaml'
[0:53:06.270216794] [5365]  INFO Camera camera_manager.cpp:284 libcamera v0.1.0+118-563cd78e
[0:53:06.313781116] [5383]  WARN RPiSdn sdn.cpp:39 Using legacy SDN tuning - please consider moving SDN inside rpi.denoise
[0:53:06.316379102] [5383]  INFO RPI vc4.cpp:444 Registered camera /base/soc/i2c0mux/i2c@1/ov5647@36 to Unicam device /dev/media1 and ISP device /dev/media0
[0:53:06.316441212] [5383]  INFO RPI pipeline_base.cpp:1142 Using configuration file '/usr/share/libcamera/pipeline

In [4]:
res = (160, 100)
preview_config = picam2.create_preview_configuration(main={"size": res}, lores={"size": res, "format": "YUV420"})
picam2.configure(preview_config)
stride = picam2.stream_configuration("lores")["stride"]
picam2.start()

[0:53:07.136295444] [5365]  INFO Camera camera.cpp:1183 configuring streams: (0) 160x100-XBGR8888 (1) 160x100-YUV420 (2) 1920x1080-SGBRG10_CSI2P
[0:53:07.136825868] [5383]  INFO RPI vc4.cpp:608 Sensor: /base/soc/i2c0mux/i2c@1/ov5647@36 - Selected sensor format: 1920x1080-SGBRG10_1X10 - Selected unicam format: 1920x1080-pGAA


In [8]:
buffer = picam2.capture_buffer("lores")
grey = buffer[:stride*res[1]].reshape((res[1], stride))[:, :res[0]]
input_data = np.expand_dims(grey, axis=0)
input_data = np.expand_dims(input_data, axis=-1)

In [13]:
tf_model_path = '../model_export/q-int8_d-2bit-octree.tflite'
interpreter = Interpreter(tf_model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]
input_scale, input_zero_point = input_details["quantization"]
input_tensor_index = input_details["index"]

In [16]:
if (input_scale, input_zero_point) != (0.0, 0):
	input_data = np.multiply(input_data, 1.0 / input_scale) + input_zero_point
input_data = input_data.astype(input_details["dtype"])
interpreter.set_tensor(0, input_data)
interpreter.invoke()
output = interpreter.get_tensor(output_details["index"])
pred = output[0]

In [25]:
picam2.stop()